In [1]:
# Implementation of a simple MLP network with one hidden layer. Tested on the iris data set.
# Requires: numpy, sklearn>=0.18.1, tensorflow>=1.0

# NOTE: In order to make the code simple, we rewrite x * W_1 + b_1 = x' * W_1'
# where x' = [x | 1] and W_1' is the matrix W_1 appended with a new row with elements b_1's.
# Similarly, for h * W_2 + b_2
# for dengai

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [3]:
# make function to preprocess data
def preprocess_data(data_path, labels_path=None):
    # load data and set index to city, year, weekofyear
    df = pd.read_csv(data_path)
    
    # fill missing values
    df.fillna(method='ffill', inplace=True)

    # add labels to dataframe
    if labels_path:
        labels = pd.read_csv(labels_path)
        #df = df.join(labels)
    
    # separate san juan and iquitos
    sj_features = df[df.city == 'sj']
    iq_features = df[df.city == 'iq']
    if labels_path:
        sj_labels = labels[labels.city == 'sj']
        iq_labels = labels[labels.city == 'iq']    
        return sj_features, iq_features, sj_labels, iq_labels
    return sj_features, iq_features

In [4]:
sj_features, iq_features, sj_labels, iq_labels = preprocess_data(
                                                                'data/dengue_features_train.csv',
                                                                labels_path="data/dengue_labels_train.csv")

In [5]:
#load final test data
sj_test_final, iq_test_final = preprocess_data("data/dengue_features_test.csv")

In [6]:
#dropping date and city as city already divided

iq_features = iq_features.drop(iq_features.columns[[0,3]], axis=1)
sj_features = sj_features.drop(sj_features.columns[[0,3]], axis=1)
sj_test_final = sj_test_final.drop(sj_test_final.columns[[0,3]], axis=1)
iq_test_final = iq_test_final.drop(iq_test_final.columns[[0,3]], axis=1)

#removing city, year, weekofyear from labels tables
sj_labels = sj_labels.total_cases
#sj_labels = sj_labels.set_index([0,2])
iq_labels = iq_labels.total_cases


sj_labels.head()

0    4
1    5
2    4
3    3
4    6
Name: total_cases, dtype: int64

In [7]:
#since data is linear it makes sense to separate data linearly
#split train and test data
sj_train = sj_features.head(800)
sj_train_target = sj_labels.head(800)
sj_test = sj_features.tail(sj_features.shape[0] - 800)
sj_test_target = sj_labels.tail(sj_labels.shape[0] - 800)



iq_train = iq_features.head(400)
iq_train_target = iq_labels.head(400)
iq_test = iq_features.tail(iq_features.shape[0] - 400)
iq_test_target = iq_labels.tail(iq_labels.shape[0] - 400)

In [8]:
#randomly separating data
# splitting data into training set and validation set

sj_train, sj_test, sj_train_target, sj_test_target = train_test_split(sj_features, sj_labels, test_size=0.2, random_state=41)

iq_train, iq_test, iq_train_target, iq_test_target = train_test_split(iq_features, iq_labels, test_size=0.2, random_state=41)

In [9]:
sj_train.head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
75,1991,41,0.09300,0.093000,0.145083,0.168167,32.10,299.562857,299.671429,295.887143,...,55.10,80.515714,32.10,17.344286,2.257143,27.657143,6.614286,31.7,22.8,8.3
119,1992,33,0.16395,0.126650,0.215057,0.219271,65.55,299.215714,299.221429,296.007143,...,81.80,82.750000,65.55,17.415714,1.957143,27.671429,7.557143,31.7,22.8,41.9
882,2007,16,0.06890,0.044267,0.108357,0.085029,0.00,299.335714,299.557143,294.507143,...,2.22,74.832857,0.00,15.854286,3.000000,27.728571,8.014286,33.3,22.8,1.8
319,1996,25,0.07125,0.064600,0.146100,0.156171,47.93,298.490000,298.564286,295.435714,...,73.70,83.322857,47.93,16.810000,1.928571,26.442857,5.800000,30.6,22.8,84.6
693,2003,35,0.07260,0.100000,0.204371,0.160357,98.68,300.668571,300.728571,297.048571,...,50.86,80.797143,98.68,18.571429,2.542857,27.914286,6.457143,31.7,23.3,69.6


In [10]:
sj_train_target.head()

75     116
119     30
882      4
319      6
693     32
Name: total_cases, dtype: int64

In [11]:
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

In [12]:
def forwardprop(X, w_1, w_2, w_3):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    h2 = tf.nn.sigmoid(tf.matmul(h, w_2))
    yhat = tf.matmul(h2, w_3)  # The \varphi function
    return yhat

In [13]:
# Layer's sizes San Juan
x_size = sj_train.shape[1]   # Number of input nodes
h_size = 256                # Number of hidden nodes
y_size = 1   # Number of outcomes (3 iris flowers)

In [14]:
# Symbols
X = tf.placeholder("float", shape=[None, x_size])
y = tf.placeholder("float", shape=[None, y_size])

In [15]:
# Weight initializations
w_1 = init_weights((x_size, h_size))
w_2 = init_weights((h_size, h_size))
w_3 = init_weights((h_size, y_size))

In [16]:
# Forward propagation
yhat    = forwardprop(X, w_1, w_2, w_3)
predict = tf.to_int64(yhat)

In [22]:
# Backward propagation
cost    = tf.losses.mean_squared_error(labels=y, predictions=yhat)
updates = tf.train.GradientDescentOptimizer(0.00000003).minimize(cost)

In [18]:
# transposing labels so that it works in tensorflow
sj_train_target = sj_train_target.as_matrix()
iq_train_target = iq_train_target.as_matrix()

sj_train_target = sj_train_target[:, None]
iq_train_target = iq_train_target[:, None]

sj_test_target = sj_test_target.as_matrix()
iq_test_target = iq_test_target.as_matrix()

sj_test_target = sj_test_target[:, None]
iq_test_target = iq_test_target[:, None]


In [19]:
#sj_train_target

In [25]:
# Run SGD
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [26]:
for epoch in range(180):
        # Train with each example
        for i in range(len(sj_train)):
            sess.run(updates, feed_dict={X: sj_train[i: i + 1], y: sj_train_target[i: i + 1]})

        train_MAE = metrics.mean_absolute_error(sj_train_target,
                                                     sess.run(predict, feed_dict={X: sj_train}))
        test_MAE  = metrics.mean_absolute_error(sj_test_target,
                                                     sess.run(predict, feed_dict={X: sj_test}))

        if epoch % 10 == 0 or epoch == 179:
            print("Epoch = %d, train loss = %.2f, test loss = %.2f"
                  % (epoch + 1, train_MAE, test_MAE))
#run model on final test data
sj_predictions = sess.run(predict, feed_dict={X: sj_test_final})

Epoch = 1, train loss = 34.18, test loss = 29.26
Epoch = 11, train loss = 32.33, test loss = 27.41
Epoch = 21, train loss = 31.46, test loss = 26.51
Epoch = 31, train loss = 30.67, test loss = 25.72
Epoch = 41, train loss = 29.94, test loss = 24.99
Epoch = 51, train loss = 28.71, test loss = 23.85
Epoch = 61, train loss = 28.16, test loss = 23.32
Epoch = 71, train loss = 27.67, test loss = 22.84
Epoch = 81, train loss = 27.24, test loss = 22.44
Epoch = 91, train loss = 26.86, test loss = 22.09
Epoch = 101, train loss = 26.52, test loss = 21.80
Epoch = 111, train loss = 26.23, test loss = 21.59
Epoch = 121, train loss = 26.00, test loss = 21.43
Epoch = 131, train loss = 25.81, test loss = 21.31
Epoch = 141, train loss = 25.67, test loss = 21.24
Epoch = 151, train loss = 25.60, test loss = 21.27
Epoch = 161, train loss = 25.60, test loss = 21.24
Epoch = 171, train loss = 25.56, test loss = 21.31
Epoch = 180, train loss = 25.57, test loss = 21.40


In [27]:
sess.close()

In [28]:
#run model again but for Iquitos, no need to redo x_size since have same number of variables


In [29]:
# reset session
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [30]:
for epoch in range(180):
        # Train with each example
        for i in range(len(iq_train)):
            sess.run(updates, feed_dict={X: iq_train[i: i + 1], y: iq_train_target[i: i + 1]})

        train_MAE = metrics.mean_absolute_error(iq_train_target,
                                                     sess.run(predict, feed_dict={X: iq_train}))
        test_MAE  = metrics.mean_absolute_error(iq_test_target,
                                                     sess.run(predict, feed_dict={X: iq_test}))

        if epoch % 10 == 0 or epoch == 179:
            print("Epoch = %d, train loss = %.2f, test loss = %.2f"
                  % (epoch + 1, train_MAE, test_MAE))
#run model on final test data
iq_predictions = sess.run(predict, feed_dict={X: iq_test_final})

Epoch = 1, train loss = 7.33, test loss = 8.49
Epoch = 11, train loss = 6.80, test loss = 8.06
Epoch = 21, train loss = 6.69, test loss = 7.91
Epoch = 31, train loss = 6.69, test loss = 7.91
Epoch = 41, train loss = 6.69, test loss = 7.91
Epoch = 51, train loss = 6.69, test loss = 7.91
Epoch = 61, train loss = 6.69, test loss = 7.91
Epoch = 71, train loss = 6.69, test loss = 7.91
Epoch = 81, train loss = 6.69, test loss = 7.91
Epoch = 91, train loss = 6.27, test loss = 7.43
Epoch = 101, train loss = 6.26, test loss = 7.41
Epoch = 111, train loss = 6.25, test loss = 7.41
Epoch = 121, train loss = 6.25, test loss = 7.41
Epoch = 131, train loss = 6.25, test loss = 7.41
Epoch = 141, train loss = 6.25, test loss = 7.41
Epoch = 151, train loss = 6.25, test loss = 7.41
Epoch = 161, train loss = 6.25, test loss = 7.41
Epoch = 171, train loss = 6.25, test loss = 7.41
Epoch = 180, train loss = 6.24, test loss = 7.38


In [31]:
sess.close()

In [ ]:
#compile into submissions format

In [32]:
submission = pd.read_csv("data/dengue_labels_test.csv",
                         index_col=[0, 1, 2])

submission.total_cases = np.concatenate([sj_predictions, iq_predictions])
submission.to_csv("submission_MLP.csv")


In [ ]:
#submission